In [8]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
from flask import Flask, render_template, request

In [9]:
# ----- Setup -----
lemmatizer = WordNetLemmatizer()

with open("intents.json") as file:
    data = json.load(file)

words = pickle.load(open("words.pkl", "rb"))
topics = pickle.load(open("topics.pkl", "rb"))
model = load_model(r"chatbot_Model.h5")


In [10]:
def clean_up_sentences(userString):
    sentence = nltk.word_tokenize(userString)
    sentence = [lemmatizer.lemmatize(word.lower()) for word in sentence]
    return sentence


In [11]:
def bag_of_words(userString):
    sentence = clean_up_sentences(userString)
    bag = [0] * len(words)
    for w in sentence:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

In [12]:
def chatBot(userString):
    try:
        currentBag = bag_of_words(userString)
        result = model.predict(np.array([currentBag]))[0]
        # print("Result: ", result, "\n")

        # get the highest numerical value prediction
        result_index = np.argmax(result)
        # print("Highest Result Index: ", result_index, "\n")

        # get the topic associated to the highest numerical value prediction
        topic = topics[result_index]

        # IF: percentage is greater then 75%
        if result[result_index] > 0.80:
            # we grab the list of responds and choose one at random
            for tg in data["intents"]:
                if tg["tag"] == topic:
                    responses = tg['responses']
            return (random.choice(responses))

        # ELSE: we print out a confused bot response
        else:
            return ("Sorry, I didn' understand. Try asking again.")
    except Exception as e:
        return (e)


In [13]:
chatBot('hello there, beautiful')

'Hi.  What can I do for you?'

In [15]:
app = Flask(__name__)

@app.route("/help", methods=["POST"])
def help_view():
    message = request.json
    q = message['question']
    result = chatBot(q)
    return { 'status': 'OK', 'data': result }


In [ ]:
app.run(port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Feb/2022 11:27:19] "POST /help HTTP/1.1" 200 -


In [ ]:
http://127.0.0.1:5001/help